In [ ]:
def get_links_recursive(url, base_url, visited, depth):
    if depth == 0:
        return
    if url in visited:
        return
    print("Scanning:", url)
    visited.append(url)

    try:
        response = requests.get(url)
        if response.status_code != 200:
            return
    except requests.exceptions.RequestException:
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            parsed_url = urlparse(href)
            current_base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
            if not parsed_url.netloc or current_base_url == base_url:
                next_url = urljoin(base_url, href)
                next_url = next_url.split("#")[0].lower()
                if next_url.endswith('php'):
                    get_links_recursive(next_url, base_url, visited, depth - 1)

def crawl_subdomain(start_url, depth):
    parsed_start_url = urlparse(start_url)
    base_url = f"{parsed_start_url.scheme}://{parsed_start_url.netloc}{parsed_start_url.path.rsplit('/', 1)[0]}/"
    visited = list()
    get_links_recursive(start_url, base_url, visited, depth)
    return visited